# EMNIST

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import numpy as np
import pandas as pd

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [3]:
train_dataset = datasets.EMNIST(root="./data", train=True, download=True, transform=transform, split='mnist')
test_dataset = datasets.EMNIST(root="./data", train=False, download=True, transform=transform, split='mnist')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 562M/562M [00:02<00:00, 276MB/s]


In [4]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1,6 , kernel_size=5, stride=1, padding=2)
    self.conv2 = nn.Conv2d(6,16, kernel_size=5, stride=1)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.max_pool2d(x , kernel_size=2, stride=2)
    x = torch.relu(self.conv2(x))
    x = torch.max_pool2d(x, kernel_size=2, stride=2)
    x = x.view(-1, 16 * 5 * 5)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [5]:
import torch.optim as optim
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01 , momentum=0.9)


In [6]:
def train(model, train_loader, optimizer, criteriion, epoch):
  model.train()
  for batch_idx , (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    outputs = model(data)
    loss = criteriion(outputs, target)
    loss.backward()
    optimizer.step()

    if batch_idx % 100 == 0:
      print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

In [7]:
def test(model, test_loader, criteriion):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      outputs = model(data)
      test_loss += criteriion(outputs, target).item()

      pred = outputs.argmax(dim=1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")


In [8]:
num_epoch = 25
for epoch in range(1, num_epoch+1):
  train(model, train_loader, optimizer, criterion, epoch)
  test(model, test_loader, criterion)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300291
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.065202
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.172151
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.027714
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.119257
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.258293
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.020740
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.159783
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.029073
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.005500

Test set: Average loss: 0.0009, Accuracy: 9816/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.011403
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.027255
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.016722
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.011984
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.005967
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.075334
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.262756
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.046330
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.003228
T